In [ ]:
#READ THIS:!!!!!!!!
#Read Real Stock Testing TODO List.md for more information on what to do next.

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

# SETTINGS

tickers = input("Enter ticker symbols separated by commas (e.g., AAPL,MSFT): ").upper().split(',')
start_date = input("Enter the start date (YYYY-MM-DD): ")
end_date = input("Enter the end date (YYYY-MM-DD): ")
short_window = int(input("Enter the window size for short-term SMA/EMA: "))
long_window = int(input("Enter the window size for long-term SMA/EMA: "))
use_ema = input("Use EMA instead of SMA? (True/False): ").strip().lower() == 'true'
starting_cash = float(input("Enter starting capital (e.g., 100000): "))
shares_per_trade = int(input("Enter number of shares per buy/sell (e.g., 100): "))
total_realized_profit = 0
ending_cash = starting_cash
all_results = []




def backtest_crossover(ticker):
    global total_realized_profit, ending_cash
   

    print(f"\nDownloading data for {ticker}...")
    data = yf.download(ticker, start=start_date, end=end_date)

    if data.empty:
        print(f"No data found for {ticker}. Skipping.\n")
        return

    prices = data['Close'].squeeze()

    # Calculate moving averages
    if use_ema:
        short_ma = prices.ewm(span=short_window).mean()
        long_ma = prices.ewm(span=long_window).mean()
    else:
        short_ma = prices.rolling(window=short_window).mean()
        long_ma = prices.rolling(window=long_window).mean()
    
    # Calculates RSI
    delta = prices.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    rsiwindow = int(input("Enter the window size for RSI calculation (e.g., 14 STD): "))
    avgain = gain.rolling(window=rsiwindow).mean()
    avgloss = loss.rolling(window=rsiwindow).mean()
    #rsiaplha = 100 - (100/(1+avgain/avgloss)) #SMA
    rsidelta = 100 - (100/(1+gain.ewm(alpha=1/rsiwindow).mean()/loss.ewm(alpha=1/rsiwindow).mean()))
    # RSI overbought/oversold levels
    rsi_overbought = 70
    rsi_oversold = 30
    

    # Generate signals: Buy, Sell, Hold
    signals = ['Hold']
    for i in range(1, len(prices)):
        if short_ma.iloc[i] > long_ma.iloc[i] and short_ma.iloc[i-1] <= long_ma.iloc[i-1]:
            signals.append('Buy')
        elif short_ma.iloc[i] < long_ma.iloc[i] and short_ma.iloc[i-1] >= long_ma.iloc[i-1]:
            signals.append('Sell')
        else:
            signals.append('Hold')

    # Simulate trades
    position = 0
    cash = ending_cash
    portfolio_values = []
    buy_signals_dates = []
    buy_signals_prices = []
    sell_signals_dates = []
    sell_signals_prices = []
    buy_trades = 0
    sell_trades = 0
    profit = 0
    cost_basis = 0

    for i in range(len(prices)):
        price = prices.iloc[i]
        signal = signals[i]

        if signal == 'Buy' and position == 0:
            cost = price * shares_per_trade
            if cash >= cost:
                cash -= cost
                position = shares_per_trade
                cost_basis = price
                buy_trades += 1
                buy_signals_dates.append(prices.index[i])
                buy_signals_prices.append(price)
            else:
                # Not enough cash to buy
                signals[i] = 'Hold'

        elif signal == 'Sell' and position > 0:
            cash += price * position
            profit += (price - cost_basis) * position
            position = 0
            sell_trades += 1
            sell_signals_dates.append(prices.index[i])
            sell_signals_prices.append(price)

        # Track portfolio value each day
        portfolio_value = cash + position * price
        portfolio_values.append(portfolio_value)

    # Finalize if holding position at the end
    if position > 0:
        final_price = prices.iloc[-1]
        cash += final_price * position
        profit += (final_price - cost_basis) * position
        sell_signals_dates.append(prices.index[-1])
        sell_signals_prices.append(final_price)
        position = 0
        sell_trades += 1
        portfolio_values[-1] = cash
    total_realized_profit += profit
    ending_cash = cash

    roi_percent = (profit / (shares_per_trade * cost_basis)) * 100 if cost_basis > 0 else 0

    all_results.append({
        'Ticker': ticker,
        'Buy Trades': buy_trades,
        'Sell Trades': sell_trades,
        'Ending Cash': cash,
        'Realized Profit': profit,
        'ROI (%)': roi_percent
    })

    # Plotting
    # Create subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

    # --- Price and MAs ---
    ax1.plot(prices.index, prices, label='Price')
    ax1.plot(prices.index, short_ma, '--', label=f'Short {"EMA" if use_ema else "SMA"} ({short_window})')
    ax1.plot(prices.index, long_ma, '--', label=f'Long {"EMA" if use_ema else "SMA"} ({long_window})')
    ax1.scatter(buy_signals_dates, buy_signals_prices, marker='^', color='green', label='Buy Signal', s=100)
    ax1.scatter(sell_signals_dates, sell_signals_prices, marker='v', color='red', label='Sell Signal', s=100)
    ax1.set_title(f"{ticker} Crossover Strategy with RSI Filter")
    ax1.set_ylabel("Price ($)")
    ax1.legend()
    ax1.grid(True)

    # --- RSI ---
    ax2.plot(prices.index, rsidelta, label=f'RSI ({rsiwindow})', color='purple')
    ax2.axhline(rsi_overbought, color='red', linestyle='--', label='Overbought (70)')
    ax2.axhline(rsi_oversold, color='green', linestyle='--', label='Oversold (30)')
    ax2.set_ylabel("RSI")
    ax2.set_ylim(0, 100)
    ax2.set_xlabel("Date")
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

    # Print trade summary for this ticker
    print(f"\nSummary of trades for {ticker}:")
    print(f"Buy Trades: {buy_trades}")
    print(f"Sell Trades: {sell_trades}")
    print(f"Ending Cash: ${cash:,.2f}")
    print(f"Realized Profit: ${profit:,.2f}")



# Run strategy for each ticker
for ticker in tickers:
    backtest_crossover(ticker)

# Final summary
results_df = pd.DataFrame(all_results)
print("\nAll strategies completed.")
print("\nFinal Summary for all tickers:")
print(results_df.to_string(index=False))
print(f"\nTotal Realized Profit Across All Tickers: ${total_realized_profit:,.2f}")
print(f"Ending Cash After All Trades: ${ending_cash:,.2f}")
print("Thank you for using TradeVision!")

In [ ]:
##Current Issue RSI isnt working properly with buy/sell signals

signals = ['Hold']
    for i in range(1, len(prices)):
        if short_ma.iloc[i] > long_ma.iloc[i] and short_ma.iloc[i-1] <= long_ma.iloc[i-1]:
            if rsidelta.iloc[i] < rsi_oversold:  # Buy only if RSI indicates oversold
                signals.append('Buy')
            else:
                signals.append('Hold RSI not oversold')
        elif short_ma.iloc[i] < long_ma.iloc[i] and short_ma.iloc[i-1] >= long_ma.iloc[i-1]:
            if rsidelta.iloc[i] > rsi_overbought:  # Sell only if RSI indicates overbought
                signals.append('Sell')
            else:
                signals.append('Hold RSI not overbought')
        else:
            signals.append('Hold')